In [1]:
DATA_DIR = '../../data/'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [111]:
import pandas as pd

Срок выполнения задания – до 5 апреля включительно.

Отчет высылается по почте Зуеву Кириллу на адрес  kir2207@list.ru

В качестве темы письма указать:  «КЛ ДЗ № 3, ФИО»

Отчет должен быть в виде отдельного текста, код программы включается как приложение.

# Задание E.
Составить (с использованием любого модуля морфоанализа) программу, выполняющую извлечение словосочетаний определенного вида из заданного русскоязычного текста. Выделение словосочетаний может базироваться на локальных высоковероятных синтаксических связях (см. слайд 50 Лекции  № 7). Программа выводит все словосочетания  заданного вида/ов, встречающиеся в обрабатываемом тексте. 
Рассмотреть несколько (2-5) грамматических образцов словосочетаний, например:
* именные словосочетания (NP), в которые входят  грамматически согласованные прилагательные и существительное (и, возможно, порядковые числительные или наречие,  например: первый детальный написанный план или более сложный образец: первый детально написанный план);
* именные словосочетания, включающие несколько существительных и прилагательные (зависимые существительные входят в родительном падеже, прилагательные согласованы с соответствующими существительными, например: легкий отблеск далекого заката);
* предложные словосочетания, в которые входят именные словосочетания одного из грамматических образцов (например: в тихом летнем парке);
* глагольные группы, состоящие из глагола в личной форме  и зависимой именной группы (например: увидел большой стеклянный шар).

Протестировать программу на нескольких небольших текстах разных жанров.

Отчет:  Описание грамматических образцов извлекаемых словосочетаний и стратегии (алгоритма) их выделения; составленная и примененная программа с комментариями, результаты ее тестирования.

# Выкачиваем данные

## Художественный текст

In [4]:
with open(DATA_DIR+'art_alice_full_text.txt', 'r') as f:
    alice_txt = f.read()

In [5]:
alice_txt_by_chapter = {idx:s.strip() for idx, s in enumerate([s for s in re.split('Глава.*\n\n\n', alice_txt) if len(s)>0])}

## Научно-технический текст

"ПОСОБИЕ ПО НАПИСАНИЮ
РАЗНОГО РОДА ДЕЛОВЫХ ТЕКСТОВ"
https://www.iis.nsk.su/files/articles/sbor_kas_16_nesgovorova.pdf

In [6]:
with open(DATA_DIR+'doc_how_to_business_text.txt','r') as f:
    how_to_buis_txt = f.read()

In [7]:
def beautify_str(s):
    s = re.sub(r'Рис\. \d+\n+', '', s)
    s = re.sub('[^\.?;\n](\n)[^\n]', ' ', s)
    return s

In [8]:
alice_txt_by_chapter = {idx:beautify_str(s) for idx,s in alice_txt_by_chapter.items()}
how_to_buis_txt = beautify_str(how_to_buis_txt)

# Ищем статистику с pymorphy и считаем метрики

In [11]:
from pymystem3 import mystem

In [17]:
mstm = mystem.Mystem()

Installing mystem to /home/egor/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [150]:
mystem_dicts = pd.read_html('https://yandex.ru/dev/mystem/doc/grammemes-values-docpage/')
mystem_dicts = mystem_dicts[::2]

In [176]:
mystem_dicts_d = {k:mystem_dicts[idx] for idx, k in enumerate([
    'POS','Tense','Case','Number','Mood',
    'Fullness','Suprs','Person', 'Gender', 'Aspect',
    'Voice','Animacy', 'Transitivity','Other'
])}

In [19]:
import pymorphy2

In [20]:
morph = pymorphy2.MorphAnalyzer()

In [21]:
def split_by_semisentences(txt):
    #return [x.strip(' ').strip('\n').strip('\t') for x in re.sub(r'[^\w\s\'\`]','|', txt).split('|') if x.strip(' ').strip('\n')!='']
    return [x.strip() for x in re.sub(r'[^\w\s\'\`]','|', txt).split('|') if x.strip()!='']

In [22]:
def split_by_word(txt):
    return [x.strip() for x in re.split(' ', txt) if re.match('\w', x)]

In [23]:
def construct_lexeme(w):
    p = morph.parse(w)[0]
    t = p.tag
    lxm = {'word':p.word, 'POS': t.POS}    
    lxm['animacy'] = t.animacy       # одушевленность
    lxm['aspect'] = t.aspect        # вид: совершенный или несовершенный
    lxm['case'] = t.case          # падеж
    lxm['gender'] = t.gender        # род (мужской, женский, средний)
    lxm['involvment'] = t.involvement   # включенность говорящего в действие
    lxm['mood'] = t.mood          # наклонение (повелительное, изъявительное)
    lxm['number'] = t.number        # число (единственное, множественное)
    lxm['person'] = t.person        # лицо (1, 2, 3)
    lxm['tense'] = t.tense         # время (настоящее, прошедшее, будущее)
    lxm['transitivity'] = t.transitivity  # переходность (переходный, непереходный)
    lxm['voice'] = t.voice  
    
    lxm = {k:v for k,v in lxm.items() if v is not None}
    return lxm

In [24]:
txt = split_by_semisentences(alice_txt_by_chapter[0])[0]
txt

'Алисе наскучило сидеть с сестрой без дела на берегу реки'

In [25]:
txt = 'хоть сейчас был не самый подходящий момент демонстрировать свои познания'

In [240]:
def construct_lexemes_mystem(txt):
    lexemes = mstm.analyze(txt)
    lexemes = [lxm for lxm in lexemes if lxm['text'].strip()!='']
    
    new_lexemes = []
    for lxm in lexemes:
        anl = lxm['analysis']
        if len(anl)>1:
            print(f'More then 1 analysis for lexeme {lxm}')
        gr = anl[0]['gr']
        
        gr_const, gr_var = gr.split('=')[0], gr.split('=')[1].replace('(','').replace(')','')
        variants = [set(gr_const.split(',')+x.split(',')) for x in gr_var.split('|')] if gr_var!='' else [set(gr_const.split(','))]
        
        parsed_vars = []
        for var in variants:
            parsed_var = {}
            for x in var:
                for k,v in mystem_dicts_d.items():
                    if x in v[0].values:
                        parsed_var[k]=x
                        break
            parsed_var['mystem_view'] = var
            parsed_var['word'] = lxm['text']
            parsed_vars.append(parsed_var)
        new_lexemes.append(parsed_vars)
    
    return new_lexemes

# Поиск грамматических образцов

## 1.
Именные словосочетания (NP), в которые входят  грамматически согласованные прилагательные и существительное (и, возможно, порядковые числительные или наречие,  например: первый детальный написанный план или более сложный образец: первый детально написанный план);

In [444]:
lexemes[0]

[{'POS': 'ADV', 'mystem_view': {'ADV'}, 'word': 'более'}]

In [455]:
def is_quaziadj_noun_agree(adj, n):
    for var_adj in adj:
        for var_n in n:
            if var_adj['POS']=='ADV' and var_n['POS']=='S':
                return True
            
            if ((var_adj['POS'] in ['A','ANUM','APRO']) or (var_adj.get('POS')=='V' and var_adj.get('Mood')=='прич')) and \
                (var_n['POS']=='S') and \
                (var_adj['POS'] in ['APRO'] or ('Gender' in var_n and var_adj.get('Gender')==var_n.get('Gender'))) and \
                (var_n['Number']=='мн' or (var_adj.get('Number')==var_n.get('Number'))) and \
                ('Case' in var_n and var_adj.get('Case')==var_n.get('Case')) \
            : 
                return True
    return False

In [457]:
def find_NP1(lexemes):
    groups = []
    nouns = [idx for idx, lxm in enumerate(lexemes) if idx>0 and any([var['POS']=='S' for var in lxm])]
    for noun_idx in nouns:
        noun = lexemes[noun_idx]
        first_adj_idx = noun_idx
        while True:
            new_first_adj_idx = first_adj_idx-1
            if first_adj_idx<0:
                break

            new_first_adj = lexemes[new_first_adj_idx]

            if is_quaziadj_noun_agree(new_first_adj, noun): # число род падеж
                first_adj_idx -= 1
            else:
                break
        if first_adj_idx==noun_idx:
            continue

        groups.append(f' '.join([x[0]['word'] for x in lexemes[first_adj_idx:noun_idx+1]]))
    return groups

In [462]:
txt = 'хоть сейчас был не самый подходящий момент демонстрировать свои познания'

In [463]:
lexemes = construct_lexemes_mystem(txt)

In [464]:
find_NP1(lexemes)

['самый подходящий момент', 'свои познания']